In [60]:
import pandas as pd 
import math
import re
from fuzzywuzzy import fuzz

In [61]:
file_path ="../Data/processed/1924_28_removed.xlsx"

In [62]:
party_mapping = r"../Data/party_mapping.xlsx"
vote_mapping = r"../Data/vote_mapping.xlsx"

In [63]:
class ReadCsvFiles:
    def __init__(self,file_path,file_type="csv"):
        if file_type=="csv":
            self.data_df = pd.read_csv(file_path, delimiter='\t')
            self.data_list = self.data_df.values.tolist()
        elif file_type=="xlsx":
            self.data_df = pd.read_excel(file_path,header=None)
            self.data_list = self.data_df.values.tolist()
        elif file_type=="pdf":
            self.data_df = read_pdf(file_path, pages="3",multiple_tables=True,area=(400,40,100,100))
    
    def getPdfData(self):
        return self.data_df
    
    def getCsvData(self):
        return self.data_df,self.data_list

In [64]:
data,party_map_list = ReadCsvFiles(party_mapping,"xlsx").getCsvData()
data,vote_map_list = ReadCsvFiles(vote_mapping,"xlsx").getCsvData()
data,file_list = ReadCsvFiles(file_path,"xlsx").getCsvData()


In [65]:
party = {}
vote = {}
for party_name in party_map_list:
    #print(party_name[0]," ",party_name[1])
    party[party_name[0]] = party_name[1]

for vote_name in vote_map_list:
    #print(party_name[0]," ",party_name[1])
    vote[vote_name[0]] = vote_name[1]

def checkVote(name):
    for ori_vote in vote:
        if fuzz.ratio(name.strip().lower(),ori_vote.strip().lower())>95:
            return vote[ori_vote]
    return ""

def checkParty(name):
    for ori_party in party:
        if fuzz.ratio(name.strip().lower(),ori_party.strip().lower())>95:
            return party[ori_party]
    return ""

In [66]:
# replace party
for i in range(1,len(file_list)):
    if i%300==0:
        print("I:",i)
    if i>=2466:
        break

    for j in range(0,1):
        if isinstance(file_list[i][j], float) and math.isnan(file_list[i][j]):
            continue
        change = checkParty(file_list[i][j])
        if change!="":
            file_list[i][j] = change
            #print("Change:",change)
# replace votes
for i in range(1,len(file_list)):
    if i%300==0:
        print("I:",i)
    if i>2466:
        break
    for j in range(2,len(file_list[0])):
        change = checkVote(file_list[i][j])
        if change!="":
            file_list[i][j] = change
            #print("Change:",change)


        

I: 300
I: 600
I: 900
I: 1200
I: 1500
I: 1800
I: 2100
I: 2400
I: 300
I: 600
I: 900
I: 1200
I: 1500
I: 1800
I: 2100
I: 2400


In [67]:
df= pd.DataFrame(file_list)
df.to_excel("../Data/processed/1924_28_mapped.xlsx",index=False) 